# oak-d × spectacularAI × nerfstudio Google Colab ワークフロー

このノートブックは、oak-dで撮影したデータ（spectacularAI形式）をGoogle Colabにアップロードし、nerfstudioで学習・可視化・ダウンロードするまでの一連の手順をまとめたものです。

---

## 概要
1. **Colab環境セットアップ** - PyTorch(GPU対応)のインストールとGPU確認
2. **データアップロード** - processed.zipをアップロード・展開
3. **nerfstudioインストール** - nerfstudioのインストールと動作確認
4. **データ確認** - アップロードしたデータの構造確認
5. **学習実行** - nerfstudioでの学習とGPU使用状況監視
6. **結果ダウンロード** - 学習済みモデル(.ckpt)のダウンロード

## 前提条件
- ローカルで `sai-cli process --format nerfstudio` により processed/ フォルダが作成済み
- processed.zip ファイルが準備済み

In [ ]:
# 1. Colab環境セットアップ
print("=== Python・GPU環境確認 ===")
!python --version
!nvidia-smi  # GPU確認

print("\n=== パッケージインストール ===")
!pip install --upgrade pip

# GPU対応PyTorchのインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# nerfstudioのインストール
!pip install nerfstudio

print("\n=== GPU利用可能性チェック ===")
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print("✅ GPU学習が利用可能です")
else:
    print("❌ GPUが利用できません（CPUで学習されます）")

In [ ]:
# 2. データをGoogle Colabにアップロード
# processed/ フォルダをzipファイルにしてアップロードしてください
# ローカルで: processed フォルダを右クリック → 圧縮 → processed.zip

from google.colab import files
import zipfile
import os
import shutil

# zipファイルをアップロード
print("processed.zip ファイルをアップロードしてください（NerfStudio形式のデータ）")
uploaded = files.upload()

# zipファイルを展開
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"{filename} を展開しました")
        os.remove(filename)  # zipファイルを削除

# Windowsパス区切り文字の問題を修正
print("パス区切り文字を修正中...")
if os.path.exists('processed'):
    # transforms.jsonのパス区切り文字を修正
    transforms_path = 'processed/transforms.json'
    if os.path.exists(transforms_path):
        import json
        with open(transforms_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # すべてのパスを修正
        if 'frames' in data:
            for frame in data['frames']:
                if 'file_path' in frame:
                    # バックスラッシュをスラッシュに変換
                    frame['file_path'] = frame['file_path'].replace('\\', '/')
        
        # 修正したJSONを保存
        with open(transforms_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2)
        print("✓ transforms.json のパスを修正しました")

# アップロードされたディレクトリ構造を確認
print("\n=== アップロード確認 ===")
!ls -la
print("\n=== processed/ 内容確認 ===")
!ls -la processed/

In [ ]:
# 3. nerfstudio動作確認
print("=== nerfstudio動作確認 ===")

# nerfstudioのインポートテスト
import nerfstudio
print("✓ nerfstudio が正常にインポートできました")

# バージョン確認
!pip show nerfstudio | grep Version

# コマンドライン確認
print("\n=== nerfstudioコマンド確認 ===")
!ns-train --help | head -5

In [ ]:
# 4. アップロードしたデータの確認
print("=== アップロードデータ構造確認 ===")
!find processed -type f | head -20

print("\n=== 必須ファイル確認 ===")
import os

# transforms.jsonの存在確認
if os.path.exists('processed/transforms.json'):
    print("✓ transforms.json が見つかりました")
else:
    print("❌ transforms.json が見つかりません")

# 画像フォルダの確認
if os.path.exists('processed/images'):
    image_count = len([f for f in os.listdir('processed/images') if f.endswith('.jpg')])
    print(f"✓ 画像フォルダに {image_count} 枚の画像があります")
    
    if image_count > 0:
        print("✅ 学習準備完了！")
    else:
        print("❌ 画像ファイルがありません")
else:
    print("❌ images フォルダが見つかりません")

In [ ]:
    # 5. Nerfstudio学習＋リアルタイムログ表示
    import subprocess
    import sys

    print("=== 学習開始 ===")
    print("⚠️ 学習には数分〜数十分かかります\n")

    cmd = [
        "ns-train",
        "nerfacto",
        "--pipeline.datamanager.data", "processed/",
        "--output-dir", "outputs/"
    ]

    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )

    try:
        for line in process.stdout:
            print(line, end="")
            sys.stdout.flush()
    except KeyboardInterrupt:
        print("\n⚠️ 学習を中断しました")
        process.terminate()

    process.wait()

    print("\n✅ 学習コマンドが完了しました")
    print("ログは outputs/train.log にも保存可能です")


In [ ]:
# 5-2. 学習完了監視
import os
import glob
import time
from IPython.display import clear_output

def check_training_status():
    print("=== 学習状況確認 ===")
    
    # outputs/フォルダの存在確認
    if os.path.exists('outputs/'):
        print("✓ outputs/ フォルダが作成されました")
        
        # チェックポイントファイルの確認
        ckpt_files = glob.glob('outputs/**/*.ckpt', recursive=True)
        if ckpt_files:
            print(f"✓ {len(ckpt_files)} 個のチェックポイントファイルが見つかりました")
            for file in ckpt_files[-2:]:  # 最新2つを表示
                print(f"  {file}")
            return True  # 学習完了
        else:
            print("⚠️ チェックポイントファイルがまだ見つかりません")
    else:
        print("⚠️ outputs/ フォルダがまだ作成されていません")
        print("学習がまだ開始されていない可能性があります")
    
    return False  # まだ学習中

print("=== 学習完了まで自動監視開始 ===")
print("学習が完了するまで30秒間隔で確認します（最大30分間）")

# 学習完了まで監視（30秒間隔で最大60回=30分間）
max_checks = 60
for i in range(max_checks):
    clear_output(wait=True)
    print(f"📊 学習監視中... ({i+1}/{max_checks}) - 経過時間: {i*0.5:.1f}分")
    
    if check_training_status():
        print("\n🎉 学習が完了しました！")
        break
    
    if i < max_checks - 1:  # 最後のチェックでない場合
        print(f"\n⏰ 次のチェックまで30秒待機中...")
        time.sleep(30)
else:
    print("\n⚠️ 監視時間が終了しました。手動で確認してください。")

# 最終的なディレクトリ構造表示
if os.path.exists('outputs/'):
    print("\n=== 学習結果ファイル一覧 ===")
    !find outputs -type f | head -20

In [ ]:
# 5-1. 学習後のGPU使用状況確認
import torch

print("=== 学習後のGPU使用状況確認 ===")

if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"Current GPU memory used: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**3:.2f} GB")
    
    # GPU使用状況詳細
    print("\n=== GPU詳細情報 ===")
    !nvidia-smi
else:
    print("❌ CUDA が利用できません")

In [ ]:
# 6. 学習結果のダウンロード
print("=== 学習結果のダウンロード ===")
print("💾 ファイルはブラウザのダウンロードフォルダに保存されます")

from google.colab import files
import glob
import os

# outputs/ ディレクトリ内の全体構造を確認
print("\n📁 学習結果のディレクトリ構造:")
!find outputs -type f | head -20

# 重要ファイルの確認と統計
ckpt_files = glob.glob('outputs/**/*.ckpt', recursive=True)
config_files = glob.glob('outputs/**/config.yml', recursive=True)
json_files = glob.glob('outputs/**/*.json', recursive=True)

print(f"\n📊 ファイル統計:")
print(f"  チェックポイント(.ckpt): {len(ckpt_files)} 個")
print(f"  設定ファイル(.yml): {len(config_files)} 個")
print(f"  JSONファイル(.json): {len(json_files)} 個")

# チェックポイントファイルのダウンロード
if ckpt_files:
    print(f"\n📥 学習済みモデル(.ckpt)をダウンロード中...")
    for file in ckpt_files:
        if os.path.isfile(file):
            print(f"  ダウンロード: {os.path.basename(file)}")
            files.download(file)
else:
    print("\n❌ .ckptファイルが見つかりませんでした")

# 設定ファイルのダウンロード
if config_files:
    print(f"\n📥 設定ファイル(config.yml)をダウンロード中...")
    for file in config_files:
        if os.path.isfile(file):
            print(f"  ダウンロード: {os.path.basename(file)}")
            files.download(file)

# JSONファイルのダウンロード
if json_files:
    print(f"\n📥 JSONファイルをダウンロード中...")
    for file in json_files[:5]:  # 最大5個まで
        if os.path.isfile(file):
            print(f"  ダウンロード: {os.path.basename(file)}")
            files.download(file)

print(f"\n✅ ダウンロード完了！")
print(f"💡 ローカルでnerfstudio viewerを使って結果を確認できます")